# importing modules

In [1]:
import os
import sys
import shutil
import logging
import argparse
import re

from sys import argv
import numpy as np
import matplotlib.pyplot as plt

import ase
from ase import Atoms
from ase.io import read, write
from ase.build import bulk
from ase.visualize import view
from ase.neighborlist import neighbor_list
from glob import glob
import pandas as pd
from scipy.integrate import trapezoid
import diptest
import os
import csv

# function to obtain the correct epsilon_p from borges et al data

In [2]:
def extract_number_after_inl(path):
    """
    Extracts and returns the number following 'INL/' in the specified path.
    
    Parameters:
        path (str): The file path from which to extract the number.
    
    Returns:
        int: The number found after 'INL/'.
    
    Raises:
        ValueError: If no number is found after 'INL/' in the path.
        Exception: For any other unexpected errors.
    """
    try:
        # Regex to find 'INL' followed by any digits
        match = re.search(r'INL/(\d+)', path)

        # Check if a match was found
        if match:
            return int(match.group(1))  # Return the number as an integer
        else:
            # If no match is found, raise an exception
            raise ValueError("No number found after 'INL' in the path.")

    except ValueError as ve:
        # Raise the error to be handled by the caller
        raise ve

    except Exception as e:
        # Raise any other unexpected error to be handled by the caller
        raise Exception("An unexpected error occurred: " + str(e))

# converting chemical formula into an array of numbers
For example: Cr10Zr40 is converted to [0.2, 0, 0, 0, 0, 0, 0, 0, 0, 0.8]

In [3]:
# Function to parse a formula and return counts of specified elements
def parse_formula(formula, elements):
    # Dictionary to store counts of elements in the formula
    element_count = {el: 0 for el in elements}
    # Regex to find elements and their counts
    matches = re.findall(r'([A-Z][a-z]*)(\d*)', formula)
    for match in matches:
        element, count = match
        if element in element_count:
            element_count[element] = int(count) if count else 1
    # Return counts in the order of the elements list
    return [element_count[el] for el in elements]


# Plotting DOS

In [4]:
def plot_dos(energy, total_dos, d_ldos, chemFormula, output_file=None):
    """
    Plot the density of states (DOS).
    """

    #calculating where is pseudogap
    fermiIndex=np.argmin(np.abs(energy))
    dosmaxIndex1, dosmaxIndex2 = np.argmax(d_ldos[:fermiIndex]), np.argmax(d_ldos[fermiIndex:])+fermiIndex
    dosminIndex=np.argmin(d_ldos[dosmaxIndex1:dosmaxIndex2])
    pgIndex=dosminIndex+dosmaxIndex1
    #calculating g_occ/g_bond
    #g_occ=np.sum(d_ldos[:fermiIndex])
    #g_bond=np.sum(d_ldos[:pgIndex])
    g_occ=trapezoid(d_ldos[:fermiIndex], x=energy[:fermiIndex])
    g_bond=trapezoid(d_ldos[:pgIndex], x=energy[:pgIndex])

    #calculating hartigan's dip
    dip, pval = diptest.diptest(d_ldos)

    plt.figure(figsize=(8, 6))
    plt.plot(energy , total_dos, label="Total DOS", color="k",alpha=0.2,linestyle=':')
    plt.plot(energy , d_ldos, label="d LDOS", color="blue")
    plt.fill_between(energy[:fermiIndex] , d_ldos[:fermiIndex], color="blue", alpha=0.5, label="$g_{occ}$")
    plt.fill_between(energy[fermiIndex:pgIndex] , d_ldos[fermiIndex:pgIndex], color="red", alpha=0.5, label="$g_{bond}$")
    plt.title('{:s}: $\\frac{{g_{{occ}}}}{{g_{{bond}}}} = {:0.3f} ~;~ dip= {:0.3f}$'.format(chemFormula,g_occ/g_bond,dip))
    #plt.plot(energy)

    plt.axvline(0, color='brown', linestyle='--', label="Fermi Level")  # Fermi level at 0
    plt.axvline(energy[pgIndex],color='black', linestyle=':', label="Pseudo gap")
    plt.xlabel("Energy (eV)")
    plt.ylabel("Density of States (states/eV/atom)")
    #plt.title("Density of States (DOS)")
    plt.legend()
    plt.grid(alpha=0.5)

    if output_file:
        plt.savefig(output_file, dpi=300)
    plt.close()

    return(g_occ,g_bond,dip)

# Plotting displacements

In [5]:
def plotDisplacements(ai,chemSpecies,d,D,contcar):
    #contcar has complete path to contcar
    dir=contcar.rpartition('/')[0]
    #print(dir)
    if dir=='':
        outputFigure='displacements.png'
        outputCSV='displacements.csv'
        outputIndex='u_vs_index.png'
    else:
        outputFigure=dir+'/displacements.png'
        outputCSV=dir+'/displacements.csv'
        outputIndex=dir+'u_vs_index.png'
    #plotting histogram of atomic displacements
    plt.figure()
    plt.hist(d)
    #print("mu={:0.2f} sigma={:0.2f}".format(np.mean(d),np.std(d)))
    plt.ylabel('number of atoms')
    plt.xlabel('$d$ [$\mathrm{\AA}$]')
    plt.title("Histogram of atomic displacements\n $\mu=${:0.2f} $\sigma=${:0.2f}".format(d.mean(),d.std()))
    plt.savefig(outputFigure)
    plt.close()

    #plotting individual displacements
    plt.figure(figsize=(7, 4))
    plt.plot(np.arange(1,len(D)+1),D[:,0],label='x',c='k',marker='o')
    plt.plot(np.arange(1,len(D)+1),D[:,1],label='y',c='r',marker='s')
    plt.plot(np.arange(1,len(D)+1),D[:,2],label='z',c='b',marker='d')
    #plt.xlim(0,73)
    plt.ylim(-0.4,0.4)
    plt.ylabel('$\\Delta u$')
    plt.xlabel('atom index')
    plt.legend(frameon=0)
    plt.savefig(outputIndex)
    plt.close()

    combinedData=np.transpose([ai,chemSpecies,d,D[:,0],D[:,1],D[:,2]]) #(np.shape(ai),np.shape(aj),np.shape(d),np.shape(D))
    df = pd.DataFrame(combinedData)
    #combinedData=np.concatenate((ai,aj,d,D), axis=1)
    # Save the array as a CSV file
    #np.savetxt(outputCSV, combinedData, delimiter=",",fmt='%.1f,%s,%.2f,%.2f,%.2f,%.2f')#, fmt=["%2d","%2s","%0.8e","%0.8e","%0.8e","%0.8e"])
    df.to_csv(outputCSV, index=False,header=['atom','type','|d|','dx','dy','dz'])
    return()

# Parcing DOSCAR

In [6]:
def parse_doscar(filename):
    """
    Parse the DOSCAR file and extract energies and DOS data.
    """
    try:
        with open(filename, 'r') as file:
            lines = file.readlines()

        # Skip header lines and extract the number of atoms
        num_atoms = int(lines[0].split()[0])
        emax=float(lines[5].split()[0])
        emin=float(lines[5].split()[1])
        nedos=int(lines[5].split()[2])

        # Fermi energy is in the 6th line
        e_fermi = float(lines[5].split()[3])

        # DOS data starts after 6 lines of header + 1 line for energy range
        dos_data_start = 6
        dos_data_stop = 6 + nedos
        dos_data = np.loadtxt(lines[dos_data_start:dos_data_stop])

        #partial dos of d states
        for n in range(num_atoms):
            dos_data_start = dos_data_stop+1
            dos_data_stop = dos_data_start+nedos
            ldos_data=np.loadtxt(lines[dos_data_start:dos_data_stop])
            if n==0:
                d_ldos=np.sum(ldos_data[:, -5:], axis=1)
            else:
                d_ldos+=np.sum(ldos_data[:, -5:], axis=1)

        # Extract energies and DOS
        energy = dos_data[:, 0]-e_fermi  # First column is energy
        total_dos = dos_data[:, 1]/num_atoms  # Second column is total DOS
        integrated_dos = dos_data[:, 2]/num_atoms  # Third column is integrated DOS
        d_ldos=d_ldos/num_atoms # d states of all atoms

        return energy, total_dos, d_ldos
    
    except IOError:
        print(f"Failed to read file: {filename}")
        return None
    except Exception as e:
        print(f"Error parsing DOSCAR: {e}")
        return None

# Classical definition of $L_{2,1}$ norm:

Consider a matrix $X=x_{ij}$, with $x_i$ rows and $x_j$ columns 

$L_{2,1}$ norm of a matrix $X_{m \times n}$ with m rows (# samples) and n columns (# features) is defined as following:

$||X||_{2,1} = \sum_{i=1}^m \sqrt{\sum_{j=1}^n x^2_{ij}}$

Norm of all features or row norm of ith sample, $ |x_i|= \sqrt{\sum_{j=1}^n x^2_{ij}}$

$||X||_{2,1} = \sum_{i=1}^m |x_i|$

# Made up (by Krishna) definition of $L_{1,2}$ norm:

$L_{1,2}$ norm of a matrix $X_{m \times n}$ with m rows (# samples) and n columns (# features) is defined as following: 

$||X||_{1,2} = \sum_{j=1}^n \sqrt{\sum_{i=1}^m x^2_{ij}}$

Norm of jth feature of all samples or column norm of jth feature, $ |x_j|= \sqrt{\sum_{j=1}^m x^2_{ij}}$

$||X||_{1,2} = \sum_{j=1}^n |x_j|$

In [7]:
def norm(matrix):
    # Calculate the L2 norm of each row
    row_norms = np.linalg.norm(matrix, axis=1)

    # Calculate the L2 norm of each column
    col_norms = np.linalg.norm(matrix, axis=0)

    # Sum the row norms to get the L2,1 norm
    return np.sum(row_norms), np.sum(col_norms) 

# Calculate $delta$ valence electron concentration, $\delta_{VEC}$

$VEC = \sum_i c_i VEC_i$

$\delta_{VEC} = \sqrt{\sum_ic_i\left( 1-\frac{VEC_i}{VEC} \right)^2}$

$c_i$ is the atomic fraction of element $i$
$VEC_i$ is the valence electron concentration of element $i$

In [8]:
def calculateVEC_rev1(chemSpecies):
    valenceElectrons={\
        ('Cr',6), ('Ti',4), ('V',5), ('W',6), \
        ('Nb',5), ('Zr',4), ('Ta',5), ('Mo',6),\
        ('Al',3), ('Hf',4), ('Re',7) \
        } #valence electrons
    natm=len(chemSpecies)
    sum=0
    for key,value in valenceElectrons: 
        sum += chemSpecies.count(key) * value
    VEC=sum/natm
    sum=0
    for key,value in valenceElectrons: 
        atomicFraction=chemSpecies.count(key)/natm
        sum += atomicFraction*(1-value/VEC)**2
    deltaVEC=100*np.sqrt(sum)
    return(VEC,deltaVEC)

# Calculate lattice mismatch, $\delta$

$\delta = \sqrt{\sum_ic_i\left( 1-\frac{r_i}{r_\mathrm{mean}} \right)^2}$

$r_i$ is the metallic radius of element $i$

the metallic radius is obtained from https://pubs.acs.org/doi/abs/10.1021/j100785a001

A copy of the metallic radius values is available at https://en.wikipedia.org/wiki/Atomic_radii_of_the_elements_(data_page)#Note_b

$c_i$ is the atomic fraction of element $i$

$r_\mathrm{mean} = \sum_i c_i \times r_i$ is the average radius of the composition

In [9]:
def calculateLatticeMismatch(chemSpecies):
    #https://pubs.acs.org/doi/abs/10.1021/j100785a001
    #https://en.wikipedia.org/wiki/Atomic_radii_of_the_elements_(data_page)#Note_b
    metallicRadii={\
        ('Cr',1.28), ('Ti',1.47), ('V',1.34), ('W',1.39), \
        ('Nb',1.46), ('Zr',1.60), ('Ta',1.46), ('Mo',1.39),\
        ('Al',1.43), ('Hf',1.59), ('Re',1.37) \
        } #angstroms
    natm=len(chemSpecies)
    averageRadii=0
    for key,metallicRadius in metallicRadii: 
        averageRadii += metallicRadius*chemSpecies.count(key)/natm
    sum=0
    for key,metallicRadius in metallicRadii: 
        atomicFraction=chemSpecies.count(key)/natm
        sum += atomicFraction*(1-metallicRadius/averageRadii)**2
    latticeMismatch=100*np.sqrt(sum)
    return(averageRadii,latticeMismatch)

# Calculate formation enthalpy, $E_f$

$E_f = E_\mathrm{DFT} - \sum_i c_i \mu_i $

$c_i$, $\mu_i$ are the atomic fraction and chemical petential of element $i$

In [10]:
def calculateFormationEnthalpy(chemSpecies,contcar):
    #parcing outcar and getting dft energy
    outcar=contcar.replace('CONTCAR','OUTCAR') #creaing path to outcar
    # Open the file and saving the last occurrence of "free energy"
    last_line = None
    with open(outcar, 'r') as file:
        for line in file:
            if "free  energy" in line:
                E = float(line.split()[4])  # Update whenever "free  energy" is found
    #chemical potential = energy/atom in its ground state
    #Cr, V, W = BCC and Ti=?
    chemicalPotentials={\
        ('Cr',-19.01176346/2), ('Ti',-15.52404561/2), ('V',-17.88015161/2), ('W',-26.02666999/2), \
        ('Hf',-19.91764361/2), ('Mo',-21.89263846/2), ('Nb',-20.18494537/2), ('Re',-24.85173447/2), \
        ('Ta',-23.72258787/2), ('Zr',-17.04089628/2) \
        } #eV/atom
    natm=len(chemSpecies)
    totalChemicalPotential=0
    for key,value in chemicalPotentials: 
        totalChemicalPotential += chemSpecies.count(key) * value
    formationEnthalpy=(E-totalChemicalPotential)/natm
    return(formationEnthalpy)

# (newer function) Calculate formation enthalpy, $E_f$

$E_f = E_\mathrm{DFT} - \sum_i c_i \mu_i $

$c_i$, $\mu_i$ are the atomic fraction and chemical petential of element $i$

In [11]:
def calculateFormationEnthalpy_rev1(chemSpecies, contcar):
    try:
        # Replacing CONTCAR with OUTCAR in the path to get the correct file
        outcar = contcar.replace('CONTCAR', 'OUTCAR')
        
        # Initialize variables
        last_line = None
        E = None
        
        # Read the OUTCAR file and extract the last occurrence of "free energy"
        with open(outcar, 'r') as file:
            for line in file:
                if "free  energy   TOTEN" in line:  # Make sure to match the exact phrase!
                    E = float(line.split()[4])

        # Handle the case where E could not be found
        if E is None:
            logging.error(f"No 'free energy' found in {outcar}")
            return None
        
        # Define chemical potentials in eV/atom
        chemicalPotentials={\
            ('Cr',-19.01176346/2), ('Ti',-15.52404561/2), ('V',-17.88015161/2), ('W',-26.02666999/2), \
            ('Hf',-19.91764361/2), ('Mo',-21.89263846/2), ('Nb',-20.18494537/2), ('Re',-24.85173447/2), \
            ('Ta',-23.72258787/2), ('Zr',-17.04089628/2) \
            } #eV/atom

        # Calculate the total chemical potential for the species
        natm=len(chemSpecies)
        totalChemicalPotential=0
        for key,value in chemicalPotentials: 
             totalChemicalPotential += chemSpecies.count(key) * value

        # Calculate formation enthalpy per atom
        formationEnthalpy = (E - totalChemicalPotential) / natm
        
        return formationEnthalpy

    except FileNotFoundError:
        logging.error(f"File not found: {outcar}")
        return None
    except ValueError:
        logging.error(f"Error processing values from {outcar}")
        return None
    except Exception as e:
        logging.error(f"An unexpected error occurred: {e}")
        return None

# Calculate supercell size based on lattice constants and number of atoms
calculating average lattice constant using, $a_0 = \frac{V}{N/2}$

V is the total volume of the supercell

N is the total number of atoms in the supercell and N/2 (for bcc) is the toal number of unitcells

$supercell ~size = approximate \left( \frac{[a,b,c]}{a_0} \right)$

a, b, c are the lattice constants of the supercell

In [12]:
def getSize(atoms):
    natm = len(atoms)
    cell = atoms.get_cell()[:]
    V = atoms.get_volume()
    a0 = (V / (natm / 2)) ** (1 / 3)
    size = np.rint(cell / a0)
    size = np.array(size, dtype=int)
    return size

# Calculate LLD

In [13]:
def getAtomicDisplacements(atoms,contcar):

    #calculating supercell size
    sc=getSize(atoms)
    chemSpecies=atoms.get_chemical_symbols()

    #creating ideal bcc supercell with relaxed dimensions
    relaxedCell=atoms.get_cell()[:]
    natm=len(atoms)
    bcc_uc = bulk('Cr', 'bcc', a=1,cubic=1)
    bcc_sc = bcc_uc.repeat((sc[0,0],sc[1,1],sc[2,2]))
    symmetric_positions=bcc_sc.get_scaled_positions()
    bcc_sc.set_cell(relaxedCell)
    bcc_sc.set_scaled_positions(symmetric_positions)

    #shifting the center of mass of ideal cell to match with relaxed structure
    #ideal_com=get_com(bcc_sc,sc)
    #print("  ideal_com before resetting =",ideal_com)
    #relaxed_com=get_com(atoms,sc)
    #bcc_sc.set_positions(bcc_sc.get_positions()+relaxed_com-ideal_com)
    #ideal_com=get_com(bcc_sc,sc)

    #print("  ideal_com after  resetting =",ideal_com)
    #print("                 relaxed_com =",relaxed_com)

    #checking if the center of mass of both cells match
    #disp=relaxed_com-ideal_com
    #if (np.sqrt(disp[0]**2+disp[1]**2+disp[2]**2)>1E-8):
    #    print(np.sqrt(disp[0]**2+disp[1]**2+disp[2]**2))
    #    exit("center of mass is not matched\nsomething is wrong\nexiting")
    
    #calculating rcut
    relaxed_cell=atoms.get_cell()[:]
    rcut=(np.sqrt(3)/4)*(relaxed_cell[0,0]/sc[0,0]+relaxed_cell[1,1]/sc[1,1]+relaxed_cell[2,2]/sc[2,2])/3
    #print("rcut=",rcut)

    #combining Atoms objects to identify the ideals coordinates as first neirest neighbors
    combined=atoms+bcc_sc

    #calculating atomic displacements
    ai,aj,d,D = neighbor_list('ijdD', combined, rcut)
    #collecting only first half of the displacements
    #because we combined atoms+bcc_sc
    ai=ai[:natm]
    aj=aj[:natm]
    d=d[:natm]
    D=D[:natm]

    #block to plot histogram of displacements and
    #to identify which species has highest atomic displacements based on j
    plotDisplacements(ai,chemSpecies,d,D,contcar)

    #checking if only first nearest neighbor atoms are identified
    coordination = np.bincount(ai)
    if (np.sum(coordination)==len(atoms)):
        #print("Identified the centrosymmetric positions")
        dummy=1
    else:
        print(coordination)
        exit("Exiting. Decrease rcut value in the code")

    return(ai,d,D)

# Write the data to a CSV file

In [14]:
def write_csv(file_path,data):
    #'formation enthalpy, E_f', usually takes second place but omitted for now
    header=['chemical formula','formation enthalpy, E_f','lattice constant, a0',\
          'lattice mismatch, delta','valence electron concentration, vec',\
          'average displacement, <u>','displacement l21 norm, l21_norm_u', '<u>/l21_norm_u',\
          'local lattice displacement, lld','total occupied states, g_occ',\
          'total bonding states, g_bond','g_occ/g_bond',"Hartigan's dip, dip", 'epsilon_p']
    
    header=(['Cr', 'Hf', 'Mo', 'Nb', 'Re', 'Ta', 'Ti', 'V', 'W', 'Zr',\
                'E_f [eV]',\
                'a_0 [Ang]',\
                'r_mean [Ang]',\
                'delta_r [%]',\
                '<|u_ij|> [Ang]',\
                '<|u_i|> [Ang]',\
                '<||u_ij||_2,1> [Ang]',\
                '<||u_ij||_1,2> [Ang]',\
                'vec [e-]',\
                'delta_vec [%]',\
                'g_occ [e-]',\
                'g_bond [e-]',\
                'BSD',\
                'dip',\
                'epsilon_p_model [%]',\
                'a0_borges [Ang]', \
                'Ecoh_borges [eV/atom]',\
                'gamma_s Borges [mJ/m2]', \
                'gamma_us Borges [mJ/m2]',\
                'VEC Borges [e-]', \
                'BSD borges', \
                'dip Borges', \
                'epsilon_p experiments'])


    try:
        with open(file_path, mode="w", newline="") as file:
            writer = csv.writer(file)

            # Write the header
            writer.writerow(header)

            # Write the data rows
            writer.writerows(data)
    except IOError as e:
        print(f"Error writing to file {file_path}: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
    return()

# Ductility model

$\varepsilon_p = -13706.12*(BSD)^3 + 38196.22*(BSD)^2 - 35519.70*(BSD) + 11025.99$

In [15]:
def ep_polymodel(BSD):
    epsilon_p=-13706.12*(BSD)**3 + 38196.22*(BSD)**2 - 35519.70*(BSD) + 11025.99
    if (epsilon_p>1):
        return( epsilon_p )
    else:
        return( None )

# Main function

In [16]:
def main():

    #reading borges data
    borges_data=pd.read_csv('Borges.txt',sep='\t')
    

    #setting header of data
    data=[]

    #collecting all contcars
    topdir='/pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/'
    contcars=glob(topdir+'**/CONTCAR',recursive=1)
    #list of elements in the dataset in the alphabetical order
    elements=['Cr', 'Hf', 'Mo', 'Nb', 'Re', 'Ta', 'Ti', 'V', 'W', 'Zr']

    #looping through all contcars
    for icontcar,contcar in enumerate(contcars):

        #not considering backup contcars
        if ('backup' in contcar):
            dummy=1
        
        #calculating doscar and plotting
        elif ('ldos' in contcar):
            dummy=2
        
        #calculating lld and plotting
        else:
            print(icontcar, contcar)
            relaxed = read(contcar) #reading contcar
            #getting chemical symbols
            chemSpecies=relaxed.get_chemical_symbols()
            chemFormula=relaxed.get_chemical_formula()

            #converting chemical symbols to array of composition
            elements_list=parse_formula(chemFormula, elements)
            sample=(elements_list/np.sum(elements_list)).tolist()

            #calculating formation enthalpy
            formationEnthalpy=calculateFormationEnthalpy_rev1(chemSpecies,contcar)

            #calculating a0
            natm=len(relaxed)
            V0=relaxed.get_volume()
            a0=(V0/(natm/2))**(1/3)

            #calculating ldos, g_occ/g_bond, dip and plotting ldos
            doscar=contcar.replace('CONTCAR','/ldos/DOSCAR') #creaing path to doscar
            ldosplot=contcar.replace('CONTCAR','ldos/ldos.png') #creating path to plot
            #calculating if DOSCAR is present at ldos/DOSCAR 
            if os.path.isfile(doscar):
                with open(doscar, 'r') as file: line_count = len(file.readlines())
                if (line_count>0):
                    energy, total_dos, d_ldos = parse_doscar(doscar) #parsing doscar
                    ##calculating descriptors and plotting
                    g_occ, g_bond, dip = plot_dos(energy, total_dos, d_ldos, chemFormula, output_file=ldosplot)
                    g_occ_over_g_bond = g_occ/g_bond
            #setting electronic descriptors to None if DOSCAR is absent at ldos/DOSCAR
            else:
                g_occ, g_bond, dip = None, None, None
                g_occ_over_g_bond= None

            #block to calculate w_VEC and valence electron concentration (VEC)
            vec, delta_vec=calculateVEC_rev1(chemSpecies)
            w_vec=vec-2
            
            #block to calculate lattice mismatch
            averageRadius,latticeMismatch=calculateLatticeMismatch(chemSpecies)

            #calculate atomic displacements
            ai,d,D=getAtomicDisplacements(relaxed,contcar)

            #lld
            average_d=np.mean(d)
            average_D=np.mean(np.abs(D))
            l21_norm, l12_norm=norm(D)
            l21_norm = l21_norm/natm
            l12_norm = l12_norm/3
            lld=w_vec*average_d/l21_norm

            #epsilon_p from borges et al data
            try:
                number = extract_number_after_inl(contcar)
                epsilonp_borges=borges_data['epsilon_p [%]'].iloc[number-1]
                gammas_borges=borges_data['gamma_s  [mJ/m2]'].iloc[number-1]
                gammaus_borges=borges_data['gamma_us [mJ/m2]'].iloc[number-1]
                Ecoh_borges=borges_data['Ecoh [eV/atom]'].iloc[number-1]
                bsd_borges=borges_data['gocc/gbond'].iloc[number-1]
                dip_borges=borges_data["dip"].iloc[number-1]
                a0_borges=borges_data["a0 [Ang]"].iloc[number-1]
                vec_borges=borges_data["VEC [e-]"].iloc[number-1]
            except Exception as e:
                print(e)


            #extending the sample to include all festures
            sample.extend([formationEnthalpy,a0,averageRadius,latticeMismatch,\
                           average_D,average_d,l21_norm,l12_norm, vec, delta_vec, \
                           g_occ, g_bond, g_occ_over_g_bond, dip, ep_polymodel(g_occ_over_g_bond),\
                           a0_borges, Ecoh_borges, gammas_borges, gammaus_borges, vec_borges, bsd_borges, dip_borges, epsilonp_borges])

            #appending the sample to the data
            data.append(sample)

    write_csv(topdir+'./data.csv',data)

    return()

# Calling the main function

In [17]:
if __name__ == "__main__":

    parser = argparse.ArgumentParser(description='Run the analysis in the directory where the subdirectories contain **/CONTCAR and **/ldos/DOCSAR files.')
    #parser.add_argument('--files', type=str, required=True, help='Path to the file containing list of OUTCAR/vasprun.xml file paths.')
    #parser.add_argument('--order', type=str, required=True, help='Path to the file specifying the atom order.')

    #args = parser.parse_args()

    main()

0 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/33/4/CONTCAR
1 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/33/1/CONTCAR
2 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/33/3/CONTCAR
3 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/33/5/CONTCAR
4 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/33/2/CONTCAR
5 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/35/4/CONTCAR
6 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/35/1/CONTCAR
7 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/35/3/CONTCAR
8 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/35/5/CONTCAR
9 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/35/2/CONTCAR
10 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/34/4/CONTCAR
11 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/34/1/CONTCAR
12

ERROR:root:No 'free energy' found in /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/31/4/OUTCAR


80 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/31/4/CONTCAR
81 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/31/1/CONTCAR
82 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/31/3/CONTCAR
83 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/31/5/CONTCAR
84 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/31/2/CONTCAR
85 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/21/4/CONTCAR
86 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/21/1/CONTCAR
87 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/21/3/CONTCAR
88 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/21/5/CONTCAR
89 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/21/2/CONTCAR
90 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/20/4/CONTCAR
91 /pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/20/1/

In [18]:
data=pd.read_csv('Borges.txt',sep='\t')
print(data)
data['epsilon_p [%]'].iloc[23]

    epsilon_p [%]  VEC [e-]  gamma_s  [mJ/m2]  gamma_us [mJ/m2]  a0 [Ang]  \
0               4      6.35              3114              1119     2.948   
1               4      4.60              2069               550     3.328   
2               1      5.60              2742              1210     3.164   
3               6      5.25              2489               664     3.133   
4               4      5.50              2615               732     3.078   
5               5      5.25              2565               685     3.130   
6              10      4.80              2304               729     3.216   
7              16      4.75              2153               682     3.267   
8              50      4.55              1978               471     3.374   
9              11      4.73              2073               541     3.335   
10             15      4.67              2051               560     3.353   
11              6      5.20              2444               572     3.283   

8

In [19]:
# The file path
path = '/pscratch/sd/k/kcpitike/MPEA/ductility_descriptor_review/borges/INL/33/4/CONTCAR'

# Regex to find 'INL' followed by any digits
match = re.search(r'INL/(\d+)', path)
print(match)

# Check if a match was found
if match:
    number = match.group(1)  # The first group captures the digits after 'INL/'
    #print("The number after 'INL' is:", number)

else:
    print("No number found after 'INL' in the path.")

<re.Match object; span=(64, 70), match='INL/33'>


In [20]:
import numpy as np
a=np.array([1,2,-3,4,-5])
print(abs(a))
print(a)

[1 2 3 4 5]
[ 1  2 -3  4 -5]
